In [21]:
import pandas as pd


In [22]:
movies = pd.read_csv("C:\\Users\\guhan\\Desktop\\Movie Recommendation System\\movies.csv")
ratings = pd.read_csv("C:\\Users\\guhan\\Desktop\\Movie Recommendation System\\ratings.csv")


In [23]:
print("Movies Data:")
print(movies.head())

Movies Data:
   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  


In [24]:
print("\nRatings Data:")
print(ratings.head())


Ratings Data:
   userId  movieId  rating   timestamp
0       1      296     5.0  1147880044
1       1      306     3.5  1147868817
2       1      307     5.0  1147868828
3       1      665     5.0  1147878820
4       1      899     3.5  1147868510


In [25]:
data = pd.merge(ratings, movies, on='movieId')

print("\nMerged Data:")
print(data.head())



Merged Data:
   userId  movieId  rating   timestamp  \
0       1      296     5.0  1147880044   
1       1      306     3.5  1147868817   
2       1      307     5.0  1147868828   
3       1      665     5.0  1147878820   
4       1      899     3.5  1147868510   

                                              title  \
0                               Pulp Fiction (1994)   
1  Three Colors: Red (Trois couleurs: Rouge) (1994)   
2  Three Colors: Blue (Trois couleurs: Bleu) (1993)   
3                                Underground (1995)   
4                        Singin' in the Rain (1952)   

                        genres  
0  Comedy|Crime|Drama|Thriller  
1                        Drama  
2                        Drama  
3             Comedy|Drama|War  
4       Comedy|Musical|Romance  


In [26]:
user_item_matrix = data.pivot_table(index='userId', columns='title', values='rating').fillna(0)

print("\nUser-Item Matrix:")
print(user_item_matrix.head())


User-Item Matrix:
title   $5 a Day (2008)  'Round Midnight (1986)  'burbs, The (1989)  \
userId                                                                
1                   0.0                     0.0                 0.0   
2                   0.0                     0.0                 0.0   
3                   0.0                     0.0                 0.0   
4                   0.0                     0.0                 0.0   
5                   0.0                     0.0                 0.0   

title   (500) Days of Summer (2009)  \
userId                                
1                               0.0   
2                               0.0   
3                               0.0   
4                               0.0   
5                               0.0   

title   (Dis)Honesty: The Truth About Lies (2015)  \
userId                                              
1                                             0.0   
2                                             0.0 

In [27]:
from sklearn.neighbors import NearestNeighbors
import numpy as np

In [28]:
item_user_matrix = user_item_matrix.T

item_user_matrix = item_user_matrix.apply(pd.to_numeric, errors='coerce').fillna(0)

model_knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=10)
model_knn.fit(item_user_matrix)

def get_item_recommendations(movie_title, n_recommendations=10):
    distances, indices = model_knn.kneighbors(item_user_matrix.loc[movie_title].values.reshape(1, -1), n_neighbors=n_recommendations+1)
    recommendations = [item_user_matrix.index[indices.flatten()[i]] for i in range(1, len(distances.flatten()))]
    return recommendations

movie_title = 'Toy Story (1995)'
recommendations = get_item_recommendations(movie_title, n_recommendations=10)
print(f"\nRecommendations for {movie_title}:")
print(recommendations)


Recommendations for Toy Story (1995):
['Star Wars: Episode IV - A New Hope (1977)', 'Star Wars: Episode VI - Return of the Jedi (1983)', 'Forrest Gump (1994)', 'Independence Day (a.k.a. ID4) (1996)', 'Lion King, The (1994)', 'Back to the Future (1985)', "Ferris Bueller's Day Off (1986)", 'Toy Story 2 (1999)', 'Finding Nemo (2003)', 'Toy Story 3 (2010)']


In [29]:
import ipywidgets as widgets
from IPython.display import display
import difflib

In [30]:
movie_title_input = widgets.Combobox(
    description="Movie Title:",
    options=sorted(movies['title'].unique()),
    ensure_option=True,
    continuous_update=True
)

button = widgets.Button(description="Get Recommendations")

output = widgets.Output()

def on_button_click(b):
    with output:
        output.clear_output()
        movie_title = movie_title_input.value
        # Case-insensitive match for the movie title
        movie_title_matches = difflib.get_close_matches(movie_title, movies['title'].unique(), n=1, cutoff=0.0)
        if movie_title_matches:
            movie_title = movie_title_matches[0]
            recommendations = get_item_recommendations(movie_title, n_recommendations=10)
            print(f"\nRecommendations for {movie_title}:")
            for rec in recommendations:
                print(rec)
        else:
            print(f"No matches found for {movie_title}.")

button.on_click(on_button_click)

display(movie_title_input, button, output)

Combobox(value='', description='Movie Title:', ensure_option=True, options=('"BLOW THE NIGHT!" Let\'s Spend th…

Button(description='Get Recommendations', style=ButtonStyle())

Output()